**Step 1: Setup TensorFlow 1.x**

In [ ]:
!pip install --upgrade tensorflow==1.14.0

In [ ]:
import tensorflow as tf
print(tf.__version__)

**Step 2: Clone the tensorflow-yolo-v3 repository**

In [ ]:
# Clone
!git clone https://github.com/mystic123/tensorflow-yolo-v3

**Step 3a: Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

**Step 3b: Navigate to tensorflow-yolo-v3 repository**

In [ ]:
%cd tensorflow-yolo-v3/

**Step 4: Download .weights and .classes files**

In [ ]:
!gdown --id <your_classes.txt_file_id>  #classes.txt
!gdown --id <your_yolov3_training_last.weights_file_id>  #yolov3_training_last.weights

**Step 5: Convert YOLOv3 .weights to .pb**

In [ ]:
!python convert_weights_pb.py --class_names "../classes.names" --weights_file "../yolov4-tiny-obj_final.weights" --data_format "NHWC" 

**Step 6a: Restart runtime**

Click on Runtime on the menu bar and select **Restart runtime.**

**Step 6b: Setup TensorFlow 2.x**

In [ ]:
%tensorflow_version 2.x

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import tempfile
import os
import glob

**Step 7a: Navigate to tensorflow-yolo-v3 repository**

In [ ]:
%cd tensorflow-yolo-v3/

**Step 7b: Check input and output nodes of neural network**


In [ ]:
import tensorflow.compat.v1 as tf
gf = tf.GraphDef()   
m_file = open('frozen_darknet_yolov3_model.pb','rb')
gf.ParseFromString(m_file.read())

with open('somefile.txt', 'a') as the_file:
     for n in gf.node:
         the_file.write(n.name+'\n')

file = open('somefile.txt','r')
data = file.readlines()
output = data[len(data)-1]
print("Output array = ", output)

file.seek ( 0 )
input=file.readline()
print("Input array = ", input)

**Step 8: Convert frozen_darknet_yolov3_model.pb to .tflite**

In [ ]:
# Reference: https://github.com/sayakpaul/Adventures-in-TensorFlow-Lite/blob/master/DeepLabV3/DeepLab_TFLite_COCO.ipynb
# Load the TensorFlow model
# The preprocessing and the post-processing steps should not be included in the TF Lite model graph 
# because some operations (ArgMax) might not support the delegates. 
# Insepct the graph using Netron https://lutzroeder.github.io/netron/
converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file='/content/tensorflow-yolo-v3/frozen_darknet_yolov3_model.pb', 
    input_arrays = ['inputs'],   # Here, 'inputs' is the value of input array from Step 7b
    output_arrays = ['output_boxes'], # Here, 'output_boxes' is the value of output array from Step 7b
    input_shapes={'inputs': [1, 416, 416, 3]} # Here, 'inputs' is the value of input array from Step 7b
)

# Optional: Perform the simplest optimization known as post-training dynamic range quantization.
# https://www.tensorflow.org/lite/performance/post_training_quantization#dynamic_range_quantization
# You can refer to the same document for other types of optimizations.
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert to TFLite Model
tflite_model = converter.convert()

_, dynamic_tflite_path = tempfile.mkstemp('.tflite')
tflite_model_size = open(dynamic_tflite_path, 'wb').write(tflite_model)
tf_model_size = os.path.getsize('/content/tensorflow-yolo-v3/frozen_darknet_yolov3_model.pb')
print('TensorFlow Model is  {} bytes'.format(tf_model_size))
print('TFLite Model is      {} bytes'.format(tflite_model_size))
print('Post training dynamic range quantization saves {} bytes'.format(tf_model_size-tflite_model_size))

In [ ]:
!ls -lh {dynamic_tflite_path}

**Step 9: Move the .tflite model into your Google Drive folder**

In [ ]:
tflite_file = ''
tflite_file = tflite_file.join(glob.glob("/tmp/*.tflite")) # Find the temp file of type .tflite
tflite_file = tflite_file[5:len(tflite_file)]  # Extract .tflite file name.

%cd ..
%cd ../tmp
os.rename(tflite_file, 'detect.tflite') # Rename the temp .tflite to detect.tflite
%mv detect.tflite ../content/gdrive/MyDrive/YOLOv3_TFLite/  # Move detect.tflite to Goolge Drive

**Step 10: Model Inference**

In [ ]:
interpreter = tf.lite.Interpreter(model_path='../content/gdrive/MyDrive/YOLOv3_TFLite/detect.tflite')

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
input_details

In [ ]:
output_details

**Step 11: Navigate to Google Drive**

In [ ]:
%cd ../content/gdrive/MyDrive/YOLOv3_TFLite/

**Step 12: Test the model**

In [ ]:
!python test_tflite.py